In [30]:
#Using google colab - importing resume.csv
from google.colab import files
uploaded = files.upload()

In [31]:
import pandas as pd
import re
import string
import nltk
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [32]:
df = pd.read_csv('Resume.csv')
df.head()
df = df[['Resume_str', 'Category']].dropna()


In [33]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

"""
this cleans the text by removing junk like HTML, emails,
and stopwords
"""
def clean_text(text):
    text = str(text)
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r'http\S+|www\S+|https\S+', '', text)
    text = re.sub(r'\S+@\S+', '', text)
    text = re.sub(r'\d+', '', text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    tokens = nltk.word_tokenize(text.lower())
    tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in stop_words and len(token) > 2]
    return ' '.join(tokens)

"""
THis pulls out wordswords like skills and
roles (nouns/adjectives) from the resume.
"""
def extract_keywords(text):
    tokens = word_tokenize(text.lower())
    tagged = pos_tag(tokens)
    keywords = [word for word, tag in tagged if tag.startswith('NN') or tag.startswith('JJ')]
    return ' '.join(keywords)

# creates common 2–3 word phrases
def generate_ngrams(text, ngram_range=(2,3)):
    try:
        vec = CountVectorizer(ngram_range=ngram_range, stop_words='english', max_features=50)
        X = vec.fit_transform([text])
        return ', '.join(vec.get_feature_names_out())
    except:
        return ''

In [34]:
df['Cleaned_Resume'] = df['Resume_str'].apply(clean_text)
df['Lower_Only'] = df['Resume_str'].str.lower()
df['Skill_NGrams'] = df['Cleaned_Resume'].apply(generate_ngrams)
df['Resume_Length'] = df['Cleaned_Resume'].apply(lambda x: len(x.split()))
df[['Resume_str', 'Cleaned_Resume']].head(5)
df.to_csv('Cleaned_Resume.csv', index=False)
print("Cleaned the resume file.")


Cleaned the resume file.


In [35]:
postings = pd.read_csv('postings.csv')
cols_to_keep = ['job_id', 'title', 'description', 'min_salary', 'max_salary', 'pay_period', 'skills_desc', 'remote_allowed']
postings = postings[cols_to_keep].dropna(subset=['title', 'description'])


#Cleaning job titles, descriptions, and skills,
def clean_text_basic(text):
    if pd.isna(text):
        return ''
    text = str(text)
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    return text.strip().lower()

postings['Cleaned_Title'] = postings['title'].apply(clean_text_basic)
postings['Cleaned_Description'] = postings['description'].apply(clean_text_basic)
postings['Job_Text'] = postings['Cleaned_Title'] + ' ' + postings['Cleaned_Description']
postings['Cleaned_Skills'] = postings['skills_desc'].fillna('').apply(clean_text_basic)
postings['Remote'] = postings['remote_allowed'].fillna(False).astype(int)

# converted salaries to yearly format
def to_yearly(row):
    factor = {'Hourly': 2080, 'Monthly': 12, 'Yearly': 1}
    pay = row['pay_period']
    return (
        row['min_salary'] * factor.get(pay, 1),
        row['max_salary'] * factor.get(pay, 1)
    )

postings[['Normalized_Min_Salary', 'Normalized_Max_Salary']] = postings.apply(to_yearly, axis=1, result_type='expand')
postings.to_csv('postings_cleaned.csv', index=False)
print("Cleaned: postings_cleaned.csv")


Cleaned: postings_cleaned.csv


In [46]:
# Adding the top 5 most common skills to each job
job_skills = pd.read_csv('job_skills_cleaned.csv')

top_skills_per_job = job_skills.groupby('job_id')['skill_abr'] \
    .apply(lambda x: list(pd.Series(x).value_counts().head(5).index)) \
    .reset_index( name='Top_Skills')

postings = pd.read_csv('postings_cleaned.csv')
postings = postings.merge(top_skills_per_job, on='job_id', how='left')
postings.to_csv('postings_enriched.csv', index=False)
print("Saved: postings_enriched.csv with top 5 skills per job")


Saved: postings_enriched.csv with top 5 skills per job


In [36]:
# Cleaned and standardized all skill names so they’re consistent
skills = pd.read_csv('skills.csv').dropna().drop_duplicates()
skills.columns = [col.strip().lower() for col in skills.columns]
skills = skills.applymap(lambda x: clean_text_basic(x) if isinstance(x, str) else x)
skills.to_csv('skills_cleaned.csv', index=False)
print("Cleaned: skills_cleaned.csv")


Cleaned: skills_cleaned.csv


/tmp/ipython-input-36-4162803854.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  skills = skills.applymap(lambda x: clean_text_basic(x) if isinstance(x, str) else x)


In [37]:
# cleaned job-skill mappings to keep only valid, readable skills so we can link jobs to the right skill sets.
job_skills = pd.read_csv('job_skills.csv').drop_duplicates()
job_skills.columns = [col.strip().lower() for col in job_skills.columns]
job_skills = job_skills[job_skills['skill_abr'].notna() & job_skills['job_id'].notna()]
job_skills['skill_abr'] = job_skills['skill_abr'].apply(clean_text_basic)

job_skills.to_csv('job_skills_cleaned.csv', index=False)
print("Cleaned: job_skills_cleaned.csv")


Cleaned: job_skills_cleaned.csv


In [38]:
# Removed duplicates and fix formatting isses
companies = pd.read_csv('companies.csv').drop_duplicates()
companies.columns = [col.strip().lower() for col in companies.columns]
if 'name' in companies.columns:
    companies['name'] = companies['name'].apply(clean_text_basic)
companies.to_csv('companies_cleaned.csv', index=False)
print("Cleaned: companies_cleaned.csv")

Cleaned: companies_cleaned.csv


In [39]:
# standardized company-to-industry mappings so we can
# accurately link companies to their sectors.
ci = pd.read_csv('company_industries.csv').drop_duplicates()
ci.columns = [col.strip().lower() for col in ci.columns]
ci.to_csv('company_industries_cleaned.csv', index=False)
print("Cleaned: company_industries_cleaned.csv")

Cleaned: company_industries_cleaned.csv


In [40]:
cs = pd.read_csv('company_specialities.csv').drop_duplicates()
cs.columns = [col.strip().lower() for col in cs.columns]
cs.to_csv('company_specialities_cleaned.csv', index=False)
print("Cleaned: company_specialities_cleaned.csv")

Cleaned: company_specialities_cleaned.csv


In [41]:
# Cleaned employee count by removing duplicates and replacing missing values
# so company size info is consistent.
import numpy as np

ec = pd.read_csv('employee_counts.csv')
ec.columns = [col.strip().lower() for col in ec.columns]
ec = ec.drop_duplicates()
ec.replace({'-': np.nan, 'unknown': np.nan}, inplace=True)
ec.to_csv('employee_counts_cleaned.csv', index=False)
print("Cleaned: employee_counts_cleaned.csv")


Cleaned: employee_counts_cleaned.csv


In [42]:
industries = pd.read_csv('industries.csv').drop_duplicates()
industries.columns = [col.strip().lower() for col in industries.columns]
col = industries.columns[0]
industries[col] = industries[col].map(clean_text_basic)
industries.to_csv('industries_cleaned.csv', index=False)
print("Cleaned: industries_cleaned.csv")


Cleaned: industries_cleaned.csv


In [43]:
ji = pd.read_csv('job_industries.csv').drop_duplicates()
ji.columns = [col.strip().lower() for col in ji.columns]
ji.to_csv('job_industries_cleaned.csv', index=False)
print("Cleaned: job_industries_cleaned.csv")

Cleaned: job_industries_cleaned.csv


In [50]:
# Removed all the symbols and converted them to just numbers
salaries = pd.read_csv('salaries.csv')
salaries.columns = [col.strip().lower() for col in salaries.columns]
salaries = salaries.replace({'\$': '', ',': ''}, regex=True)
for col in ['min_salary', 'max_salary', 'median_salary']:
    if col in salaries.columns:
        salaries[col] = pd.to_numeric(salaries[col], errors='coerce')
salaries.to_csv('salaries_cleaned.csv', index=False)
print("Cleaned: salaries_cleaned.csv")

Cleaned: salaries_cleaned.csv


In [45]:
# deduplicated benefit names so we can clearly identify what each job offers
benefits = pd.read_csv('benefits.csv')
benefits.columns = [col.strip().lower() for col in benefits.columns]
if 'benefit_name' in benefits.columns:
    benefits['benefit_name'] = benefits['benefit_name'].apply(clean_text_basic)
benefits = benefits.drop_duplicates()
benefits.to_csv('benefits_cleaned.csv', index=False)
print("Cleaned: benefits_cleaned.csv")

Cleaned: benefits_cleaned.csv


In [49]:
import os
import shutil
from glob import glob

output_folder = 'All_Cleaned_Files'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
cleaned_files = glob('*.csv')
keywords = ['cleaned', 'enriched', 'Resume_Labeled.csv', 'Master_Job_Index.csv']

for file in cleaned_files:
    if any(keyword in file for keyword in keywords):
        shutil.copy(file, os.path.join(output_folder, file))

# Everything’s formatted for direct use — just load and vectorize the cleaned CSVs.
print(f"All the files that was cleaned have been copied to the '{output_folder}' folder.")

All the files that was cleaned have been copied to the 'All_Cleaned_Files' folder.
